<a href="https://colab.research.google.com/github/Naijaoracle/data-analysis/blob/master/BRFSS_BPHIGH_likelihood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil
import json

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'behavioral-risk-factor-surveillance-system:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2192%2F3692%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240221%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240221T130843Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D928ce863e00084cd9b1a2d9b8f05d04d8008463dae65584600d455ea7a7db05a294e08671f7d6ecfc91d1872833a471661fe9f16f9039750de2381cfca0e3aee0a9e3e1dc2794688dd88d86593924f6881a638d8c4d41c56f4184a9cbb6744f751eeb65d8df08126c930959f2275555f3845d31d7c70a48024f656ab4c4da3ebf129fecb079672e119c3c51f4f5ecec456390b4655ea7e8e4ebc83a43b4c4f060f704135e1b9374c08b27cd01b116f0560c62a488e64cc3e5f858323ff1fd212e740d569d4ea68870c237ad64ace9b13dcc6b9354b7a365ba55a82392b1bbce72b2f02977911c888a4a19cf5f9edb2d1fa0519c0206a2bb9efd2fe697f2b927b,brfss2015processed:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4470260%2F7665332%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240221%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240221T130843Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3af5d63f75344645e6cf43e31734ad21d8051e93f840794f3dc7fb3ad98d399bb7066000cdf22cedfc3b571e0e12eca4c298ae3b9435b92fdf1211274f6ec1f546d485bef68394196a8e20309f06e75403b3ec539b2920be5b9eac9e1a1fab2b955248e972cfb770caea7d29e3565e71fa15d0c7f957c989c76950394c7a195f33fbd48ad80cddc7fb139355a3e1079802f8a41837362238276c7c83ca26154447fb3fb5fb56c3c374200c84a3c84130e4f3c5d36c288166ea6bee8042cc779d9e574b70613fb40ff1284c1badba95c3ef252067f4db973936d2abc0cb20d2a628ab32deafcee2837fd5b8c097fb212298448f72b19e0830406ef33a7db41af1,selcted-columns:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4474365%2F7671206%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240221%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240221T130843Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2aba78ab0f6557c6b4b8b0e23120cdba8a74d8acf4d5bbdf09d061d25f216a82bd2af2bd4cb8e7c2a312d036939578037707f2cccf4da70f43fd0599514633ee244b4354df77c4d3345509ed3947c893a1302dc2d70d02d20c56728f122bcf535595fc41acdd0d40082d3c8c7c6e2adc4190ec84027009a35956cd52ff3e58b0f89a4a485dc223b7a368b1cd47cb35fc0db48a949b229d6cfbc4f7bfb3a2470107a500813d8173908b54cdce3518908fc73c1a94119e9f96f95eaec26fe33b1dcb4978e3e624da661c23261b2188b7b89dfd396c6a62b44ef0e7321b7fd1f40efbb369083fe3401c5ad13f4c8ee32012b5d392948dfba77a31ea10c802ef8a83'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 434508654 bytes downloaded
Downloaded and uncompressed: behavioral-risk-factor-surveillance-system
[==================================================] 17187266 bytes downloaded
Downloaded and uncompressed: brfss2015processed
[==================================================] 531 bytes downloaded
Downloaded and uncompressed: selcted-columns
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/behavioral-risk-factor-surveillance-system/2012.csv
/kaggle/input/behavioral-risk-factor-surveillance-system/2013.csv
/kaggle/input/behavioral-risk-factor-surveillance-system/2011.csv
/kaggle/input/behavioral-risk-factor-surveillance-system/2015.csv
/kaggle/input/behavioral-risk-factor-surveillance-system/2015_formats.json
/kaggle/input/behavioral-risk-factor-surveillance-system/2014.csv
/kaggle/input/selcted-columns/selected_columns.json
/kaggle/input/brfss2015processed/labels.json
/kaggle/input/brfss2015processed/variable_labels.json
/kaggle/input/brfss2015processed/AIassignment.csv


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Open the JSON file for reading
with open('/content/drive/MyDrive/Projects /colab et al/SCHOOL/selected_columns.json', 'r') as file:
    columns_list = json.load(file)

print(columns_list)

['SEQNO', '_AGEG5YR', 'SEX', '_RACE', 'BPHIGH4', 'BPMEDS', '_BMI5CAT', 'CHECKUP1', 'BLOODCHO', 'TOLDHI2', 'CVDINFR4', 'CVDCRHD4', 'CVDSTRK3', 'GENHLTH', 'PHYSHLTH', 'MENTHLTH', 'POORHLTH', '_PA300R2', 'HLTHPLN1', 'MEDCOST', 'CHCSCNCR', 'CHCOCNCR', 'ADDEPEV2', 'CHCKIDNY', 'DIABETE3', 'DIABAGE2', 'MARITAL', 'EDUCA', 'INCOME2', 'WEIGHT2', 'HEIGHT3', 'PREGNANT', 'SMOKE100', 'SMOKDAY2', 'STOPSMK2', 'LASTSMK2', 'USENOW3', 'ALCDAY5', 'AVEDRNK2', 'DRNK3GE5', 'MAXDRNKS', 'FRUIT1', 'FVBEANS', 'FVGREEN', 'EXERANY2', 'EXRACT11', 'EXRACT21', 'EXEROFT2', 'STRENGTH', 'PREDIAB1', 'INSULIN', 'BLDSUGAR', 'WTCHSALT', 'LONGWTCH', 'DRADVISE', 'CVDASPRN', 'RDUCHART', 'RDUCSTRK', 'LSATISFY', 'ADPLEASR', 'ADDOWN', 'ADSLEEP', 'ADENERGY']


In [5]:
brfss_df = pd.read_csv('/kaggle/input/behavioral-risk-factor-surveillance-system/2015.csv')

In [6]:
brfss_df.head()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENUM,...,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3
0,1.0,1.0,b'01292015',b'01',b'29',b'2015',1200.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0
1,1.0,1.0,b'01202015',b'01',b'20',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,2.0,2.0,3.0,3.0,4.0,2.0,2.0,NaN,NaN,2.0
2,1.0,1.0,b'02012015',b'02',b'01',b'2015',1200.0,2.015000e+09,2.015000e+09,1.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,NaN
3,1.0,1.0,b'01142015',b'01',b'14',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,9.0
4,1.0,1.0,b'01142015',b'01',b'14',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0


In [7]:
df = brfss_df[columns_list]

In [8]:
columns_list = [col for col in columns_list if col in df.columns]
df = brfss_df[columns_list]

In [9]:
df.head()

,SEQNO,_AGEG5YR,SEX,_RACE,BPHIGH4,BPMEDS,_BMI5CAT,CHECKUP1,BLOODCHO,TOLDHI2,...,LONGWTCH,DRADVISE,CVDASPRN,RDUCHART,RDUCSTRK,LSATISFY,ADPLEASR,ADDOWN,ADSLEEP,ADENERGY
0,2.015000e+09,9.0,2.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.015000e+09,7.0,2.0,1.0,3.0,NaN,3.0,4.0,1.0,2.0,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.015000e+09,11.0,2.0,1.0,3.0,NaN,2.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.015000e+09,9.0,2.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.015000e+09,9.0,2.0,1.0,3.0,NaN,2.0,1.0,1.0,2.0,...,777.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df.columns

Index(['SEQNO', '_AGEG5YR', 'SEX', '_RACE', 'BPHIGH4', 'BPMEDS', '_BMI5CAT',
       'CHECKUP1', 'BLOODCHO', 'TOLDHI2', 'CVDINFR4', 'CVDCRHD4', 'CVDSTRK3',
       'GENHLTH', 'PHYSHLTH', 'MENTHLTH', 'POORHLTH', '_PA300R2', 'HLTHPLN1',
       'MEDCOST', 'CHCSCNCR', 'CHCOCNCR', 'ADDEPEV2', 'CHCKIDNY', 'DIABETE3',
       'DIABAGE2', 'MARITAL', 'EDUCA', 'INCOME2', 'WEIGHT2', 'HEIGHT3',
       'PREGNANT', 'SMOKE100', 'SMOKDAY2', 'STOPSMK2', 'LASTSMK2', 'USENOW3',
       'ALCDAY5', 'AVEDRNK2', 'DRNK3GE5', 'MAXDRNKS', 'FRUIT1', 'FVBEANS',
       'FVGREEN', 'EXERANY2', 'EXRACT11', 'EXRACT21', 'EXEROFT2', 'STRENGTH',
       'PREDIAB1', 'INSULIN', 'BLDSUGAR', 'WTCHSALT', 'LONGWTCH', 'DRADVISE',
       'CVDASPRN', 'RDUCHART', 'RDUCSTRK', 'LSATISFY', 'ADPLEASR', 'ADDOWN',
       'ADSLEEP', 'ADENERGY'],
      dtype='object')

In [11]:
df = pd.read_csv('/content/drive/MyDrive/Projects /colab et al/SCHOOL/AIassignment.csv')

In [12]:
df.head()

,SEQNO,_AGEG5YR,SEX,_RACE,BPHIGH4,BPMEDS,_BMI5CAT,CHECKUP1,BLOODCHO,TOLDHI2,...,LONGWTCH,DRADVISE,CVDASPRN,RDUCHART,RDUCSTRK,LSATISFY,ADPLEASR,ADDOWN,ADSLEEP,ADENERGY
0,2.015000e+09,9.0,2.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,...,777.0,7.0,7.0,7.0,7.0,7.0,77.0,77.0,77.0,77.0
1,2.015000e+09,7.0,2.0,1.0,3.0,2.0,3.0,4.0,1.0,2.0,...,777.0,2.0,7.0,7.0,7.0,7.0,77.0,77.0,77.0,77.0
2,2.015000e+09,11.0,2.0,1.0,3.0,2.0,2.0,1.0,1.0,1.0,...,777.0,7.0,7.0,7.0,7.0,7.0,77.0,77.0,77.0,77.0
3,2.015000e+09,9.0,2.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,...,777.0,2.0,7.0,7.0,7.0,7.0,77.0,77.0,77.0,77.0
4,2.015000e+09,9.0,2.0,1.0,3.0,2.0,2.0,1.0,1.0,2.0,...,777.0,2.0,7.0,7.0,7.0,7.0,77.0,77.0,77.0,77.0


In [13]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set the option to display all rows
pd.set_option('display.max_rows', None)

# Calculate the correlation of each feature with 'BPHIGH4'
correlation = df.corr()['BPHIGH4']

# Drop the 'BPHIGH4' itself from the series
correlation = correlation.drop('BPHIGH4')

# Sort the values in descending order
sorted_correlation = correlation.sort_values(ascending=False)

print(sorted_correlation)

BPMEDS      0.780351
DIABETE3    0.237683
TOLDHI2     0.237168
DIABAGE2    0.224232
INSULIN     0.178411
BLDSUGAR    0.160270
CHECKUP1    0.146232
BLOODCHO    0.110336
EDUCA       0.106518
LASTSMK2    0.106301
PHYSHLTH    0.096312
CHCOCNCR    0.093541
SEQNO       0.092601
CVDINFR4    0.082321
CVDSTRK3    0.081367
CHCSCNCR    0.078243
SMOKDAY2    0.074824
LONGWTCH    0.066503
HLTHPLN1    0.066225
ADDEPEV2    0.064614
POORHLTH    0.062278
MARITAL     0.061440
CHCKIDNY    0.059596
RDUCHART    0.054913
SMOKE100    0.054773
RDUCSTRK    0.052840
DRADVISE    0.051081
_RACE       0.049698
CVDCRHD4    0.049188
WTCHSALT    0.046864
HEIGHT3     0.038157
ADENERGY    0.031221
ADSLEEP     0.025030
CVDASPRN    0.022729
ADPLEASR    0.020300
ADDOWN      0.019466
USENOW3     0.019038
WEIGHT2     0.011408
SEX         0.010277
MEDCOST     0.008819
INCOME2     0.008590
PREGNANT    0.006399
PREDIAB1    0.001914
EXERANY2   -0.001929
_PA300R2   -0.003841
FRUIT1     -0.005793
MENTHLTH   -0.006053
FVBEANS    -0

In [14]:
pd.reset_option('display.max_rows')


In [15]:
df.head()

,SEQNO,_AGEG5YR,SEX,_RACE,BPHIGH4,BPMEDS,_BMI5CAT,CHECKUP1,BLOODCHO,TOLDHI2,...,LONGWTCH,DRADVISE,CVDASPRN,RDUCHART,RDUCSTRK,LSATISFY,ADPLEASR,ADDOWN,ADSLEEP,ADENERGY
0,2.015000e+09,9.0,2.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,...,777.0,7.0,7.0,7.0,7.0,7.0,77.0,77.0,77.0,77.0
1,2.015000e+09,7.0,2.0,1.0,3.0,2.0,3.0,4.0,1.0,2.0,...,777.0,2.0,7.0,7.0,7.0,7.0,77.0,77.0,77.0,77.0
2,2.015000e+09,11.0,2.0,1.0,3.0,2.0,2.0,1.0,1.0,1.0,...,777.0,7.0,7.0,7.0,7.0,7.0,77.0,77.0,77.0,77.0
3,2.015000e+09,9.0,2.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,...,777.0,2.0,7.0,7.0,7.0,7.0,77.0,77.0,77.0,77.0
4,2.015000e+09,9.0,2.0,1.0,3.0,2.0,2.0,1.0,1.0,2.0,...,777.0,2.0,7.0,7.0,7.0,7.0,77.0,77.0,77.0,77.0


In [16]:
# Create a contingency table
contingency_table = pd.crosstab(df['BPHIGH4'], df['BPMEDS'])

print(contingency_table)

from tabulate import tabulate

print(tabulate(contingency_table, headers='keys', tablefmt='psql', showindex="always"))

BPMEDS      1.0     2.0  7.0  9.0
BPHIGH4                          
1.0      149034   28833  258   63
2.0           0    3271    0    0
3.0           0  254319    0    0
4.0           0    4312    0    0
7.0           0     862    0    0
9.0           0     504    0    0
+-----------+--------+--------+-------+-------+
|   BPHIGH4 |    1.0 |    2.0 |   7.0 |   9.0 |
|-----------+--------+--------+-------+-------|
|         1 | 149034 |  28833 |   258 |    63 |
|         2 |      0 |   3271 |     0 |     0 |
|         3 |      0 | 254319 |     0 |     0 |
|         4 |      0 |   4312 |     0 |     0 |
|         7 |      0 |    862 |     0 |     0 |
|         9 |      0 |    504 |     0 |     0 |
+-----------+--------+--------+-------+-------+


In [17]:
import json


with open('/content/drive/MyDrive/Projects /colab et al/SCHOOL/labels.json','r') as f:
    labels = json.load(f)

with open('/content/drive/MyDrive/Projects /colab et al/SCHOOL/variable_labels.json','r') as file:
    key_descriptions = json.load(file)


In [18]:
print(len(labels))
print(len(key_descriptions))
print(len(df))

62
62
441456


In [19]:
mismatch = [i for i in df.columns if i not in labels]
print(mismatch)

['SEQNO']


In [20]:
print(df.columns)

Index(['SEQNO', '_AGEG5YR', 'SEX', '_RACE', 'BPHIGH4', 'BPMEDS', '_BMI5CAT',
       'CHECKUP1', 'BLOODCHO', 'TOLDHI2', 'CVDINFR4', 'CVDCRHD4', 'CVDSTRK3',
       'GENHLTH', 'PHYSHLTH', 'MENTHLTH', 'POORHLTH', '_PA300R2', 'HLTHPLN1',
       'MEDCOST', 'CHCSCNCR', 'CHCOCNCR', 'ADDEPEV2', 'CHCKIDNY', 'DIABETE3',
       'DIABAGE2', 'MARITAL', 'EDUCA', 'INCOME2', 'WEIGHT2', 'HEIGHT3',
       'PREGNANT', 'SMOKE100', 'SMOKDAY2', 'STOPSMK2', 'LASTSMK2', 'USENOW3',
       'ALCDAY5', 'AVEDRNK2', 'DRNK3GE5', 'MAXDRNKS', 'FRUIT1', 'FVBEANS',
       'FVGREEN', 'EXERANY2', 'EXRACT11', 'EXRACT21', 'EXEROFT2', 'STRENGTH',
       'PREDIAB1', 'INSULIN', 'BLDSUGAR', 'WTCHSALT', 'LONGWTCH', 'DRADVISE',
       'CVDASPRN', 'RDUCHART', 'RDUCSTRK', 'LSATISFY', 'ADPLEASR', 'ADDOWN',
       'ADSLEEP', 'ADENERGY'],
      dtype='object')


In [21]:
# Get counts by category in 'BPHIGH4' column
category_counts = df['BPHIGH4'].value_counts()

print(category_counts)

3.0    254319
1.0    178188
4.0      4312
2.0      3271
7.0       862
9.0       504
Name: BPHIGH4, dtype: int64


In [22]:
df.head()

,SEQNO,_AGEG5YR,SEX,_RACE,BPHIGH4,BPMEDS,_BMI5CAT,CHECKUP1,BLOODCHO,TOLDHI2,...,LONGWTCH,DRADVISE,CVDASPRN,RDUCHART,RDUCSTRK,LSATISFY,ADPLEASR,ADDOWN,ADSLEEP,ADENERGY
0,2.015000e+09,9.0,2.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,...,777.0,7.0,7.0,7.0,7.0,7.0,77.0,77.0,77.0,77.0
1,2.015000e+09,7.0,2.0,1.0,3.0,2.0,3.0,4.0,1.0,2.0,...,777.0,2.0,7.0,7.0,7.0,7.0,77.0,77.0,77.0,77.0
2,2.015000e+09,11.0,2.0,1.0,3.0,2.0,2.0,1.0,1.0,1.0,...,777.0,7.0,7.0,7.0,7.0,7.0,77.0,77.0,77.0,77.0
3,2.015000e+09,9.0,2.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,...,777.0,2.0,7.0,7.0,7.0,7.0,77.0,77.0,77.0,77.0
4,2.015000e+09,9.0,2.0,1.0,3.0,2.0,2.0,1.0,1.0,2.0,...,777.0,2.0,7.0,7.0,7.0,7.0,77.0,77.0,77.0,77.0


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


In [24]:
# Define features (X) and target variable (y)
X = df.drop(columns=['BPHIGH4'])  # Exclude the target variable 'BPHIGH4'
y = df['BPHIGH4']  # Target variable 'BPHIGH4'

In [25]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
%%timeit
# Train the classifier
rf_classifier.fit(X_train, y_train)

In [ ]:
# Predict the target variable
y_pred = rf_classifier.predict(X_test)

In [ ]:
# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
# Get the predicted probabilities
y_pred_prob = rf_classifier.predict_proba(X_test)

# Binarize the output
y_test_bin = label_binarize(y_test, classes=np.unique(y_test))

# Now you can calculate the ROC AUC score
roc_auc = roc_auc_score(y_test_bin, y_pred_prob, multi_class='ovr')


In [ ]:
# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC Score:", roc_auc)

In [ ]:
import lightgbm as lgb
# Define the classifier
lgb_clf = lgb.LGBMClassifier(objective='multiclass', num_class=len(np.unique(y)), random_state=42)

In [ ]:
# Fit the model
lgb_clf.fit(X_train, y_train)

In [ ]:
# Predict the classes
y_pred_lgb = lgb_clf.predict(X_test)

In [ ]:
# Binarize the output
y_test_bin = label_binarize(y_test, classes=np.unique(y_test))
y_pred_bin_lgb = label_binarize(y_pred_lgb, classes=np.unique(y_test))

# Calculate the accuracy, Precision, recall, and F1-score
accuracy_lgb = accuracy_score(y_test, y_pred_lgb)
precision_lgb = precision_score(y_test, y_pred_lgb, average='weighted', zero_division=0)
recall_lgb = recall_score(y_test, y_pred_lgb, average='weighted', zero_division=0)
f1_lgb = f1_score(y_test, y_pred_lgb, average='weighted', zero_division=0)

In [ ]:
print("Accuracy:", accuracy_lgb)
print("Precision: ", precision_lgb)
print("Recall: ", recall_lgb)
print("F1 Score: ", f1_lgb)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Define the classifier
logreg_clf = LogisticRegression(multi_class='ovr', solver='liblinear')

In [ ]:
# Fit the model
logreg_clf.fit(X_train, y_train)

In [ ]:
# Predict the classes
y_pred_logreg = logreg_clf.predict(X_test)

In [ ]:
# Calculate the accuracy
accuracy_logreg = accuracy_score(y_test, y_pred_logreg)

In [ ]:
# Binarize the output
y_test_bin = label_binarize(y_test, classes=np.unique(y_test))
y_pred_bin_logreg = label_binarize(y_pred_logreg, classes=np.unique(y_test))

In [ ]:
# Precision, recall, and F1-score
precision_logreg = precision_score(y_test, y_pred_logreg, average='weighted', zero_division=0)
recall_logreg = recall_score(y_test, y_pred_logreg, average='weighted', zero_division=0)
f1_logreg = f1_score(y_test, y_pred_logreg, average='weighted', zero_division=0)

In [ ]:
print("Accuracy: ", accuracy_logreg)
print("Precision: ", precision_logreg)
print("Recall: ", recall_logreg)
print("F1 Score: ", f1_logreg)

In [ ]:
from sklearn import svm

# Define the classifier
svm_clf = svm.SVC(decision_function_shape='ovo')

In [ ]:
%%timeit
# Fit the model
svm_clf.fit(X_train, y_train)

In [ ]:
# Predict the classes
y_pred_svm = svm_clf.predict(X_test)

In [ ]:
# Binarize the output
y_test_bin = label_binarize(y_test, classes=np.unique(y_test))
y_pred_bin_svm = label_binarize(y_pred_svm, classes=np.unique(y_test))

In [ ]:
# Calculate the accuracy
accuracy_svm = accuracy_score(y_test, y_pred_svm)

In [ ]:
# Precision, recall, and F1-score
precision_svm = precision_score(y_test, y_pred_svm, average='weighted', zero_division=0)
recall_svm = recall_score(y_test, y_pred_svm, average='weighted', zero_division=0)
f1_svm = f1_score(y_test, y_pred_svm, average='weighted', zero_division=0)

In [ ]:
print("Accuracy: ", accuracy_svm)

In [ ]:
print("Precision: ", precision_svm)
print("Recall: ", recall_svm)
print("F1 Score: ", f1_svm)

In [ ]:
# ROC AUC score
roc_auc_svm = roc_auc_score(y_test_bin, y_pred_svm, multi_class='ovr')
print("ROC AUC Score: ", roc_auc_svm)